# ResNet

 **理解残差连接的思想
 会调用PyTorch自带的ResNet
 知道怎么改最后一层**
 
## 一、ResNet到底在干什么？

### 普通网络的问题

```
想象你在爬100层楼梯

普通网络 = 只有楼梯
    爬到第50层你就累死了
    信息（梯度）传到底层已经没力气了
    底层学不动 → 整个网络学偏

实际现象：
    20层网络比10层好
    50层网络反而比20层差了！
    不是过拟合，是根本训练不动
```

### ResNet的解决方法

```
ResNet = 楼梯 + 电梯

每隔几层就装一部电梯
就算楼梯走不通，电梯也能直达

结果：100层甚至1000层都能训练！
```

### 翻译成代码语言

```
普通网络：
    输出 = F(x)
    x经过几个层变成F(x)，完了

ResNet：
    输出 = F(x) + x      ← 就多了一个 +x ！
    
    那个 +x 就是"电梯"
    不管F(x)学没学到东西，x都能直接传过去
```

---

## 二、残差连接画成图

```
最简单的理解：

输入x ──→ 卷积 → BN → ReLU → 卷积 → BN ──→ 相加 → ReLU → 输出
  │                                              ↑
  └──────────────────────────────────────────────┘
                    这条线就是残差连接
                    x直接跳过去加到结果上
```

**就是加了一条线，让输入可以"抄近路"到输出！**

---

## 三、为什么 +x 这么厉害？

```
情况1：中间的层学到了有用的东西
    输出 = 有用的东西 + x = 比x更好 ✓

情况2：中间的层什么都没学到（输出≈0）
    输出 = 0 + x = x
    至少不会比没加这些层更差 ✓

所以：
    加层只会变好或不变
    永远不会变差！
    这就是为什么ResNet可以做到100+层
```

---

## 四、ResNet的整体结构（知道大概）

```
图片进来
    ↓
一个大卷积快速缩小图片
    ↓
一堆残差块（通道越来越多，图片越来越小）
    ↓
全局平均池化（不管图片多大，变成1×1）
    ↓
一个全连接层输出分类结果
```

**和LeNet的套路完全一样，只是中间换成了残差块！**

```
LeNet：  Conv → Pool → Conv → Pool → FC
ResNet：Conv → 残差块×N → 全局Pool → FC

核心区别就是中间用了残差块（带+x的那种）
```

---

## 五、不同版本的ResNet

```
ResNet18：18层，最快，精度一般
ResNet34：34层，常用，速度精度平衡
ResNet50：50层，更强，更慢
ResNet101：101层，刷榜用
ResNet152：152层，太贵，很少用

数字 = 卷积层的数量
数字越大 = 残差块越多 = 更深 = 更强但更慢

实际中最常用：ResNet34 或 ResNet50
```

---

## 需要会的代码

### 调用PyTorch自带的ResNet

```python
import torchvision.models as models

# 加载预训练好的ResNet（别人已经训练好的）
resnet = models.resnet18(pretrained=True)
```

**不需要自己写残差块！**

### 微调时改最后一层（这个要会！）

```python
import torchvision.models as models
import torch.nn as nn

# 第1步：加载预训练的ResNet18
resnet = models.resnet18(pretrained=True)

# 第2步：看看最后一层长什么样
print(resnet.fc)    
# 输出：Linear(in_features=512, out_features=1000)
# 默认输出1000类（ImageNet的类别数）

# 第3步：改成你需要的类别数
resnet.fc = nn.Linear(512, 10)    # 改成10类
# 或者
resnet.fc = nn.Linear(512, 2)     # 改成2类（比如猫狗分类）
```

**为什么要改最后一层？**

```
预训练的ResNet是在ImageNet上训练的（1000类）
你的任务可能只有10类或2类
所以要把最后一层的输出改成你的类别数
```

### 微调的完整流程

```python
import torchvision.models as models
import torch.nn as nn

# 1. 加载预训练模型
resnet = models.resnet18(pretrained=True)

# 2. 冻结所有层（不训练前面的层）
for param in resnet.parameters():
    param.requires_grad = False

# 3. 改最后一层（这一层要训练）
resnet.fc = nn.Linear(512, 你的类别数)
# 新创建的层默认requires_grad=True，会被训练

# 4. 搬到GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet.to(device)

# 5. 正常训练就行了
```

---

## 七、总结

### 概念理解

```
1. ResNet = 普通网络 + 残差连接（+x）
2. 残差连接让信息可以跳过某些层
3. 好处：网络再深也能训练
4. 残差连接是现代深度学习的标配（到处都用）
```

### 代码（会这些就够了）

```python
# 加载预训练ResNet
resnet = models.resnet18(pretrained=True)

# 改最后一层
resnet.fc = nn.Linear(512, 类别数)

# 冻结前面的层（微调时）
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc = nn.Linear(512, 类别数)  # 新层自动可训练
```

### 记忆

```
ResNet = 每隔几层加一条捷径（输出 = 层的结果 + 输入）
用的时候直接调用 models.resnet18/34/50
改最后一层适配你的任务
```
